<a href="https://colab.research.google.com/github/advik-7/Agents/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import os
email_user = os.environ.get("EMAIL_USER")
email_pass = os.environ.get("EMAIL_PASS")


In [19]:
import imaplib
import email
from email.header import decode_header
from datetime import datetime, timedelta
from transformers import pipeline
import os

def fetch_and_classify_emails(email_user, email_pass):
    classifier = pipeline("text-classification", model="mrm8488/bert-tiny-finetuned-enron-spam-detection")
    imap_host = "imap.gmail.com"

    try:
        mail = imaplib.IMAP4_SSL(imap_host)
        mail.login(email_user, email_pass)
        mail.select("inbox")
        date_24hrs_ago = (datetime.now() - timedelta(days=1)).strftime("%d-%b-%Y")
        status, messages = mail.search(None, f'SINCE {date_24hrs_ago}')
        if status != "OK":
            print("No messages found!")
            return

        email_ids = messages[0].split()

        for email_id in email_ids:
            status, msg_data = mail.fetch(email_id, "(RFC822)")
            if status != "OK":
                print(f"Failed to fetch email ID {email_id.decode()}")
                continue

            for response_part in msg_data:
                if isinstance(response_part, tuple):
                    msg = email.message_from_bytes(response_part[1])
                    subject, encoding = decode_header(msg["Subject"])[0]
                    if isinstance(subject, bytes):
                        subject = subject.decode(encoding if encoding else "utf-8")

                    from_ = msg.get("From")
                    body = ""
                    if msg.is_multipart():
                        for part in msg.walk():
                            content_type = part.get_content_type()
                            content_disposition = str(part.get("Content-Disposition"))
                            if content_type == "text/plain" and "attachment" not in content_disposition:
                                try:
                                    body = part.get_payload(decode=True).decode()
                                    break
                                except Exception as e:
                                    print(f"Error decoding body: {e}")
                    else:
                        try:
                            body = msg.get_payload(decode=True).decode()
                        except Exception as e:
                            print(f"Error decoding body: {e}")

                    body_truncated = body[:512]
                    classification = classifier(body_truncated)[0]
                    label = classification['label']
                    category = "Spam" if label == "LABEL_1" else "Not Spam"

                    print(f"Subject: {subject}")
                    print(f"From: {from_}")
                    print(f"Category: {category}")
                    print("-" * 50)

        mail.logout()

    except Exception as e:
        print(f"An error occurred: {e}")

email_user = os.environ.get("EMAIL_USER")
email_pass = os.environ.get("EMAIL_PASS")

if email_user and email_pass:
    fetch_and_classify_emails(email_user, email_pass)
else:
    print("Please set the EMAIL_USER and EMAIL_PASS environment variables.")


Subject: Your Jeda.ai Trial is Ending - Don't Miss Our Early Bird Launch Offer!
From: "Captain from Jeda.ai" <captain@jeda.ai>
Category: Spam
--------------------------------------------------
Subject: 📄TSPO interacts with VDAC1 and triggers a ROS-mediated inhibition of mitochondrial quality control was mentioned
From: Your Paper was Mentioned <premium@academia-mail.com>
Category: Spam
--------------------------------------------------
Subject: The ultimate creative tool kit
From: "Adobe Special Offers" <mail@mail.adobe.com>
Category: Spam
--------------------------------------------------
Subject: Take your website to the next level with custom settings
From: The HubSpot Team <thehubspotteam@hubspot.com>
Category: Spam
--------------------------------------------------
Subject: AI powered website with your logo ready in seconds
From: LOGO.com <marketing-reply@logo.com>
Category: Spam
--------------------------------------------------
Subject: Black Friday Announcement…
From: "Maxon" <